In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
import psycopg2
from sklearn.externals import joblib# better than pickle

/Users/hangli/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
# !pip install joblib --upgrade


In [4]:
connection_string = psycopg2.connect(host="mypostgresdb.cnm9emj4w74o.us-east-2.rds.amazonaws.com", port = 5432, database="project3", user="rootlh", password="password")

In [5]:
# pip install -U textblob


In [6]:
from textblob import TextBlob
# The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). 
# The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0]
# where 0.0 is very objective and 1.0 is very subjective.


In [7]:
df = pd.read_sql_query('select * from post_info', con=connection_string)
df.head()

,type,posts
0,INTJ,'https://www.youtube.com/watch?v=8IEQpfA528M ...
1,INFP,"'It's one of my daily 'music doodles', one min..."
2,INFP,'NO. But I have dyscalculia. :X|||I've learned...
3,ISTP,"'288970 It's been a while, guys. Also I bleach..."
4,INFP,'29 and still have it. Never had a girlfriend....


In [8]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
print(df.head())

   type                                              posts  words_per_comment
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86
1  INFP  'It's one of my daily 'music doodles', one min...              17.46
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90
4  INFP  '29 and still have it. Never had a girlfriend....              27.98


In [9]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
df['question_per_comment'] = df['posts'].apply(lambda x: x.count('?')/50)
df['excl_per_comment'] = df['posts'].apply(lambda x: x.count('!')/50)
df['upper_case'] = df['posts'].str.findall(r'[A-Z]').str.len()/50
# df['adj'] = df['posts'].str.findall(r'[A-Z]').str.len()/50
df[['polarity', 'subjectivity']] = df['posts'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
# df['emojis'] = df['posts'].str.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]').str.len()
df['ellipsis_per_comment'] = df['posts'].apply(lambda x: x.count('...')/50)


print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86   
1  INFP  'It's one of my daily 'music doodles', one min...              17.46   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30   
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90   
4  INFP  '29 and still have it. Never had a girlfriend....              27.98   
5  INFJ  'It gets to be pretty obvious the lifestyle di...              20.98   
6  INTP  'I used to find this difficult until I realize...              23.80   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...              33.88   
8  ENFP  'Can I just say I'm so glad and thankful that ...              26.28   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...              21.80   

   question_per_comment  excl_per_comment  upper_case  polarity  subjectivity  \
0                  0.68    

In [10]:
# df[['x']] = df['posts'].apply(lambda Text: pd.Series(TextBlob(Text).np_counts))
# df

In [11]:
df.dtypes

type                     object
posts                    object
words_per_comment       float64
question_per_comment    float64
excl_per_comment        float64
upper_case              float64
polarity                float64
subjectivity            float64
ellipsis_per_comment    float64
dtype: object

In [12]:
df

,type,posts,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,ellipsis_per_comment
0,INTJ,'https://www.youtube.com/watch?v=8IEQpfA528M ...,22.86,0.68,0.00,3.64,0.101422,0.546946,0.60
1,INFP,"'It's one of my daily 'music doodles', one min...",17.46,0.30,0.12,4.68,0.067913,0.542686,0.34
2,INFP,'NO. But I have dyscalculia. :X|||I've learned...,13.30,0.12,0.06,3.68,0.128141,0.579492,0.32
3,ISTP,"'288970 It's been a while, guys. Also I bleach...",10.90,0.16,0.26,2.50,0.052717,0.557769,0.10
4,INFP,'29 and still have it. Never had a girlfriend....,27.98,0.16,0.06,4.42,0.046999,0.505684,0.80
...,...,...,...,...,...,...,...,...,...
8670,INTJ,'What about when you hurt someone else? Do yo...,16.34,0.20,0.02,4.42,0.181248,0.573210,0.50
8671,INFJ,'Any INFJ having a relationship with a ENFP? B...,29.80,0.34,0.28,6.32,0.189571,0.557296,0.90
8672,INFJ,'That's very interesting how you're in a relat...,29.10,0.02,0.02,4.52,0.189957,0.518341,0.74
8673,ESTP,'I don't feel bad at all about that and I don'...,23.74,0.46,0.30,5.18,0.151486,0.527806,0.72


In [13]:
map1 = {"I": 0, "E": 1}
map2 = {"N": 0, "S": 1}
map3 = {"T": 0, "F": 1}
map4 = {"J": 0, "P": 1}
df['I-E'] = df['type'].astype(str).str[0]
df['I-E'] = df['I-E'].map(map1)
df['N-S'] = df['type'].astype(str).str[1]
df['N-S'] = df['N-S'].map(map2)
df['T-F'] = df['type'].astype(str).str[2]
df['T-F'] = df['T-F'].map(map3)
df['J-P'] = df['type'].astype(str).str[3]
df['J-P'] = df['J-P'].map(map4)
print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86   
1  INFP  'It's one of my daily 'music doodles', one min...              17.46   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30   
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90   
4  INFP  '29 and still have it. Never had a girlfriend....              27.98   
5  INFJ  'It gets to be pretty obvious the lifestyle di...              20.98   
6  INTP  'I used to find this difficult until I realize...              23.80   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...              33.88   
8  ENFP  'Can I just say I'm so glad and thankful that ...              26.28   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...              21.80   

   question_per_comment  excl_per_comment  upper_case  polarity  subjectivity  \
0                  0.68    

In [14]:
X = df.drop(['type','posts','I-E','N-S','T-F','J-P'], axis=1)
print(X.shape)

(8675, 7)


In [15]:
X.shape

(8675, 7)

In [16]:
df['type'].unique()

array(['INTJ', 'INFP', 'ISTP', 'INFJ', 'INTP', 'ENFP', 'ESFP', 'ENTP',
       'ENFJ', 'ESTP', 'ISFP', 'ENTJ', 'ISTJ', 'ESTJ', 'ISFJ', 'ESFJ'],
      dtype=object)

In [17]:
# !pip install tensorflow

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [19]:
# Introvert vs. Extrovert
y = df['I-E']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)


In [20]:
# df["I-E"] = df["I-E"].astype("category")


In [21]:
X_train

,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,ellipsis_per_comment
5495,25.74,0.28,0.02,4.50,0.193355,0.532972,0.62
7205,30.82,0.18,0.36,6.76,0.206285,0.601249,1.16
3889,30.08,0.08,0.08,5.74,0.101339,0.516682,0.72
94,13.90,0.28,0.14,2.40,0.077992,0.690285,0.20
6675,17.58,0.12,0.02,2.88,0.117476,0.608326,0.24
...,...,...,...,...,...,...,...
3704,21.56,0.26,0.36,5.80,0.095862,0.523073,0.54
163,27.06,0.26,0.22,5.92,0.081877,0.531350,0.72
6012,31.16,0.32,0.02,6.06,0.030651,0.505661,0.74
6558,24.32,0.18,0.00,7.22,0.106238,0.503160,0.60


In [22]:
y

0       0
1       0
2       0
3       0
4       0
       ..
8670    0
8671    0
8672    0
8673    1
8674    0
Name: I-E, Length: 8675, dtype: int64

In [23]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
scaler_filename = "scaler.save"

In [25]:
joblib.dump(X_scaler, scaler_filename) 

['scaler.save']

In [26]:
X_train_scaled

array([[0.6908993 , 0.11570248, 0.005     , ..., 0.73143713, 0.5329717 ,
        0.18674699],
       [0.82767905, 0.07438017, 0.09      , ..., 0.7409577 , 0.6012491 ,
        0.34939759],
       [0.80775444, 0.03305785, 0.02      , ..., 0.66368255, 0.51668164,
        0.21686747],
       ...,
       [0.8368336 , 0.1322314 , 0.005     , ..., 0.61163283, 0.50566111,
        0.22289157],
       [0.65266559, 0.07438017, 0.        , ..., 0.66728972, 0.50315976,
        0.18072289],
       [0.63004847, 0.1322314 , 0.04      , ..., 0.69210999, 0.53853771,
        0.21084337]])

In [27]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [28]:
encoded_y_train

array([0, 0, 0, ..., 0, 1, 0])

In [29]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [30]:
y_train_categorical

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [32]:
model = Sequential()
model.add(Dense(units=200, activation='sigmoid', input_dim=7))
model.add(Dense(units=200, activation='sigmoid'))
model.add(Dense(units=2, activation='softmax'))

In [33]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               1600      
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 42,202
Trainable params: 42,202
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
204/204 - 1s - loss: 0.5524 - accuracy: 0.7650
Epoch 2/60
204/204 - 1s - loss: 0.5502 - accuracy: 0.7687
Epoch 3/60
204/204 - 0s - loss: 0.5517 - accuracy: 0.7687
Epoch 4/60
204/204 - 0s - loss: 0.5452 - accuracy: 0.7687
Epoch 5/60
204/204 - 0s - loss: 0.5489 - accuracy: 0.7687
Epoch 6/60
204/204 - 0s - loss: 0.5436 - accuracy: 0.7687
Epoch 7/60
204/204 - 0s - loss: 0.5397 - accuracy: 0.7688
Epoch 8/60
204/204 - 0s - loss: 0.5370 - accuracy: 0.7688
Epoch 9/60
204/204 - 0s - loss: 0.5365 - accuracy: 0.7688
Epoch 10/60
204/204 - 0s - loss: 0.5348 - accuracy: 0.7690
Epoch 11/60
204/204 - 0s - loss: 0.5350 - accuracy: 0.7708
Epoch 12/60
204/204 - 0s - loss: 0.5336 - accuracy: 0.7698
Epoch 13/60
204/204 - 0s - loss: 0.5327 - accuracy: 0.7701
Epoch 14/60
204/204 - 0s - loss: 0.5342 - accuracy: 0.7698
Epoch 15/60
204/204 - 0s - loss: 0.5341 - accuracy: 0.7701
Epoch 16/60
204/204 - 0s - loss: 0.5318 - accuracy: 0.7705
Epoch 17/60
204/204 - 0s - loss: 0.5330 - accuracy: 0.7701
Epoch 

In [36]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

68/68 - 0s - loss: 0.5316 - accuracy: 0.7704
Normal Neural Network - Loss: 0.5316122174263, Accuracy: 0.770401120185852


In [37]:
model.save("model_test.h5") 
from tensorflow.keras.models import load_model
modelIE = load_model("model_test.h5")

In [38]:
X_test

,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,ellipsis_per_comment
7915,20.86,0.22,0.14,3.68,0.043637,0.594063,0.42
3839,28.34,0.18,0.12,6.48,0.150611,0.494092,1.04
8168,31.20,0.18,0.36,5.60,0.162235,0.565029,1.00
3442,30.18,0.14,0.02,5.56,0.085250,0.472184,0.68
2870,10.82,0.12,0.02,2.08,0.167263,0.531493,0.18
...,...,...,...,...,...,...,...
5892,23.22,0.18,0.26,7.20,0.056211,0.608271,0.46
8035,9.36,0.20,0.02,2.36,0.073557,0.538962,0.14
2145,16.20,0.14,0.02,7.58,0.160300,0.548074,0.20
4735,17.36,0.34,0.08,2.56,0.161578,0.507725,0.34


In [39]:
X_test_scaled[:5]

array([[0.55950458, 0.09090909, 0.035     , 0.19574468, 0.62119509,
        0.59406288, 0.12650602],
       [0.76090468, 0.07438017, 0.03      , 0.34468085, 0.69996337,
        0.4940923 , 0.31325301],
       [0.83791061, 0.07438017, 0.09      , 0.29787234, 0.70852237,
        0.56502897, 0.30120482],
       [0.81044696, 0.05785124, 0.005     , 0.29574468, 0.65183606,
        0.47218373, 0.20481928],
       [0.28917609, 0.04958678, 0.005     , 0.1106383 , 0.71222486,
        0.53149301, 0.05421687]])

In [40]:
encoded_predictions = modelIE.predict_classes(X_test_scaled[:5])


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [41]:
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [42]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [0 0 0 0 0]
Actual Labels: [0, 1, 0, 0, 0]


In [65]:
docs_new = ['i like basketball','There are a few ways of doing this, depending on what you’re trying to achieve. ','happy','good', 'very good','Wo ai zu guo tian an men ????', 'tian an men shang tai yang sheng','WARNING:tensorflow:From <ipython-input-39-1ea00abbf4bb>:1: Sequential.predict_classes (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.']

In [66]:
x= [docs_new[0]+ ' ' +docs_new[1]]
x

['i like basketball There are a few ways of doing this, depending on what you’re trying to achieve. ']

In [67]:
df_test = pd.DataFrame({'posts':docs_new})

In [68]:
df_test

,posts
0,i like basketball
1,"There are a few ways of doing this, depending ..."
2,happy
3,good
4,very good
5,Wo ai zu guo tian an men ????
6,tian an men shang tai yang sheng
7,WARNING:tensorflow:From <ipython-input-39-1ea0...


In [69]:
df_test['words_per_comment'] = df_test['posts'].apply(lambda x: len(x.split())/50)


In [70]:
df_test['question_per_comment'] = df_test['posts'].apply(lambda x: x.count('?')/50)


In [71]:
df_test['excl_per_comment'] = df_test['posts'].apply(lambda x: x.count('!')/50)


In [72]:
df_test['upper_case'] = df_test['posts'].str.findall(r'[A-Z]').str.len()/50


In [73]:
df_test[['polarity', 'subjectivity','y']] = df_test['posts'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment_assessments))
df_test

,posts,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,y
0,i like basketball,0.06,0.00,0.0,0.00,0.00,0.00,[]
1,"There are a few ways of doing this, depending ...",0.30,0.00,0.0,0.02,-0.20,0.10,"[([few], -0.2, 0.1, None)]"
2,happy,0.02,0.00,0.0,0.00,0.80,1.00,"[([happy], 0.8, 1.0, None)]"
3,good,0.02,0.00,0.0,0.00,0.70,0.60,"[([good], 0.7, 0.6000000000000001, None)]"
4,very good,0.04,0.00,0.0,0.00,0.91,0.78,"[([very, good], 0.9099999999999999, 0.78000000..."
5,Wo ai zu guo tian an men ????,0.16,0.08,0.0,0.02,0.00,0.00,[]
6,tian an men shang tai yang sheng,0.14,0.00,0.0,0.00,0.00,0.00,[]
7,WARNING:tensorflow:From <ipython-input-39-1ea0...,0.26,0.00,0.0,0.18,0.00,0.00,[]


In [74]:
# subjectivity = polarity_subjectivity[1]
# subjectivity

In [75]:
df_test['ellipsis_per_comment'] = df_test['posts'].apply(lambda x: x.count('...')/50)
# ellipsis_per_comment

In [76]:
df_test

,posts,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,y,ellipsis_per_comment
0,i like basketball,0.06,0.00,0.0,0.00,0.00,0.00,[],0.0
1,"There are a few ways of doing this, depending ...",0.30,0.00,0.0,0.02,-0.20,0.10,"[([few], -0.2, 0.1, None)]",0.0
2,happy,0.02,0.00,0.0,0.00,0.80,1.00,"[([happy], 0.8, 1.0, None)]",0.0
3,good,0.02,0.00,0.0,0.00,0.70,0.60,"[([good], 0.7, 0.6000000000000001, None)]",0.0
4,very good,0.04,0.00,0.0,0.00,0.91,0.78,"[([very, good], 0.9099999999999999, 0.78000000...",0.0
5,Wo ai zu guo tian an men ????,0.16,0.08,0.0,0.02,0.00,0.00,[],0.0
6,tian an men shang tai yang sheng,0.14,0.00,0.0,0.00,0.00,0.00,[],0.0
7,WARNING:tensorflow:From <ipython-input-39-1ea0...,0.26,0.00,0.0,0.18,0.00,0.00,[],0.0


In [77]:
Xx = df_test.drop(['posts','y'], axis=1)
Xx

,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,ellipsis_per_comment
0,0.06,0.00,0.0,0.00,0.00,0.00,0.0
1,0.30,0.00,0.0,0.02,-0.20,0.10,0.0
2,0.02,0.00,0.0,0.00,0.80,1.00,0.0
3,0.02,0.00,0.0,0.00,0.70,0.60,0.0
4,0.04,0.00,0.0,0.00,0.91,0.78,0.0
5,0.16,0.08,0.0,0.02,0.00,0.00,0.0
6,0.14,0.00,0.0,0.00,0.00,0.00,0.0
7,0.26,0.00,0.0,0.18,0.00,0.00,0.0


In [78]:
# X_scaler = MinMaxScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [79]:
#do I need to scale Xx???????
X_scaled = X_scaler.transform(Xx)
X_scaled.dtype


dtype('float64')

In [80]:
X_scaled

array([[-5.38502962e-04,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  5.89063829e-01,  0.00000000e+00,
         0.00000000e+00],
       [ 5.92353258e-03,  0.00000000e+00,  0.00000000e+00,
         1.06382979e-03,  4.41797872e-01,  1.00000000e-01,
         0.00000000e+00],
       [-1.61550889e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.17812766e+00,  1.00000000e+00,
         0.00000000e+00],
       [-1.61550889e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.10449468e+00,  6.00000000e-01,
         0.00000000e+00],
       [-1.07700592e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.25912393e+00,  7.80000000e-01,
         0.00000000e+00],
       [ 2.15401185e-03,  3.30578512e-02,  0.00000000e+00,
         1.06382979e-03,  5.89063829e-01,  0.00000000e+00,
         0.00000000e+00],
       [ 1.61550889e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  5.89063829e-01,  0.00000000e+00,
         0.0000000

In [81]:
predictions = modelIE.predict_classes(X_scaled)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0])

In [60]:
prediction_labelsrrr = label_encoder.inverse_transform(predictions)
prediction_labelsrrr

array([0, 0, 0, 0, 0, 0, 0])

In [61]:
# df["I-E"] = df["I-E"].astype("category")

In [62]:
# df_predict["predicted"] = df_predict["predicted"].astype("category")


In [63]:
# df_predict["predicted"].cat.categories = ["Candidate", "Confirmed", "False Positive"]


In [ ]:
predictions2 = modelIE.predict_classes(Xx)
predictions2

In [ ]:
import numpy as np
new_data = np.array([[0.2, 0.3, 0.4]])
new_data.dtype

In [ ]:
ri = np.array(Xx)
ri

In [ ]:
predictions3 = modelIE.predict_classes(ri)
predictions3

In [ ]:
cao = np.array(docs_new)
cao

In [ ]:
predictions4 = modelIE.predict_classes(cao)

In [ ]:
#Intuitive vs. Sensing
y = df['N-S'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
model2 = Sequential()
model2.add(Dense(units=200, activation='relu', input_dim=7))
model2.add(Dense(units=200, activation='relu'))
model2.add(Dense(units=2, activation='softmax'))

In [ ]:
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model2.summary() 

In [ ]:
model2.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = model2.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
model2.save("model_testNS.h5") 
from tensorflow.keras.models import load_model
modelNS = load_model("model_testNS.h5")

In [ ]:
a=1
b=2
c=list([a,b])

In [ ]:
c

In [ ]:
# Thinking vs. Feeling
y = df['T-F'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)


In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
model3 = Sequential()
model3.add(Dense(units=200, activation='sigmoid', input_dim=7))
model3.add(Dense(units=200, activation='sigmoid'))
model3.add(Dense(units=2, activation='softmax'))

In [ ]:
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model3.summary() 

In [ ]:
model3.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = model3.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Judging vs. Perceiving
y = df['J-P'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)


In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
model4 = Sequential()
model4.add(Dense(units=200, activation='sigmoid', input_dim=7))
model4.add(Dense(units=200, activation='sigmoid'))
model4.add(Dense(units=2, activation='softmax'))

In [ ]:
model4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model4.summary() 

In [ ]:
model4.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = model4.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
#relu: 0.598

In [ ]:
# Overall
y = df['type'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
model5 = Sequential()
model5.add(Dense(units=200, activation='sigmoid', input_dim=7))
model5.add(Dense(units=200, activation='sigmoid'))
model5.add(Dense(units=16, activation='softmax'))

In [ ]:
model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model5.summary() 

In [ ]:
model5.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = model5.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# save the model to disk
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))
 
# # some time later...
 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))

#import joblib
# # save the model to disk
# filename = 'finalized_model.sav'
# joblib.dump(model, filename)
 
# # some time later...
 
# # load the model from disk
# loaded_model = joblib.load(filename)
model.save("model_test.h5") 

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
modelIE = load_model("model_test.h5")

In [ ]:
# import pickle
# # Saving model to disk
# pickle.dump(model, open('modelIE.pkl','wb'))
# model = pickle.load(open('model.pkl','rb'))

In [ ]:
encoded_predictions = modelIE.predict_classes(['i am good'])
# prediction_labels = label_encoder.inverse_transform(encoded_predictions)